In [1]:
import tensorflow as tf
from ML100kPP import InteractionMatrix
import pandas as pd
from MovieLens1MPreprocessing import RatingMatrix_ML1M

In [2]:
ML_100K_ratingMatrix = InteractionMatrix('Drama','Comedy')#change the inputs here to change the two domains

RatingMatrix = ML_100K_ratingMatrix.RatingMatrix
domainARatingMatrix = ML_100K_ratingMatrix.domainARatingMatrix
domainBRatingMatrix = ML_100K_ratingMatrix.domainBRatingMatrix

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
DomainA_train_1, DomainA_test_1 = train_test_split(domainARatingMatrix, test_size = 0.3, random_state = 0)
DomainB_train_1, DomainB_test_1 = train_test_split(domainBRatingMatrix, test_size = 0.3, random_state = 0)

DomainA_train_2 = DomainA_train_1.applymap(lambda x: 1 if x != 0 else 0 )
DomainB_train_2 = DomainB_train_1.applymap(lambda x: 1 if x != 0 else 0 )
DomainA_test_2 = DomainA_test_1.applymap(lambda x: 1 if x != 0 else 0 )
DomainB_test_2 = DomainB_test_1.applymap(lambda x: 1 if x != 0 else 0 )

domainA_train = tf.cast(DomainA_train_2, tf.float32)
domainB_train = tf.cast(DomainB_train_2, tf.float32)
domainA_test = tf.cast(DomainA_test_2, tf.float32)
domainB_test = tf.cast(DomainB_test_2, tf.float32)

In [5]:
ML_1M_ratingMatrix = RatingMatrix_ML1M('Drama', 'Comedy')

Rating_matrix_ML1M = ML_1M_ratingMatrix.ratingMatrix
domainARatingMatrix_ML1M = ML_1M_ratingMatrix.domainARatingMatrix
domainBRatingMatrix_ML1M = ML_1M_ratingMatrix.domainBRatingMatrix

In [6]:
DomainA_train_1_ML1M, DomainA_test_1_ML1M = train_test_split(domainARatingMatrix_ML1M, test_size = 0.2, 
                                                             random_state = 0)
DomainB_train_1_ML1M, DomainB_test_1_ML1M = train_test_split(domainBRatingMatrix_ML1M, test_size = 0.2,
                                                             random_state = 0)

DomainA_train_2_ML1M = DomainA_train_1_ML1M.applymap(lambda x: 1 if x != 0 else 0 )
DomainB_train_2_ML1M = DomainB_train_1_ML1M.applymap(lambda x: 1 if x != 0 else 0 )
DomainA_test_2_ML1M = DomainA_test_1_ML1M.applymap(lambda x: 1 if x != 0 else 0 )
DomainB_test_2_ML1M = DomainB_test_1_ML1M.applymap(lambda x: 1 if x != 0 else 0 )

domainA_train_ML1M = tf.cast(DomainA_train_2_ML1M, tf.float32)
domainB_train_ML1M = tf.cast(DomainB_train_2_ML1M, tf.float32)
domainA_test_ML1M = tf.cast(DomainA_test_2_ML1M, tf.float32)
domainB_test_ML1M = tf.cast(DomainB_test_2_ML1M, tf.float32)

In [7]:
def kl_loss(mu, sigma):
    return 0.5*tf.reduce_mean(tf.reduce_sum(1+ tf.math.log(tf.square(sigma)) - tf.square(mu) - tf.square(sigma)
            , axis = 1))

In [8]:
def loss_reconstruct(matrix, recon_matrix):
        
    log_softmax_matrix = tf.nn.log_softmax(recon_matrix)
    
    nll = -tf.reduce_mean(tf.reduce_sum(log_softmax_matrix * matrix, axis = -1))
      
    return nll

In [9]:
import tensorflow as tf 
from tensorflow.keras.layers import Dense

In [10]:
class VAE(tf.keras.Model):
    
    def __init__(self, encode_dim, share_dim, z_dim, decode_dim, interaction_matrix, lambda1, lambda2, weight_decay):
        
        super().__init__()
        
        decode_dim.append(int(interaction_matrix.shape[1]))
        
        first_encode_dim = []
        
        first_encode_dim.append(int(interaction_matrix.shape[1]))
        
        encode_dim = first_encode_dim + encode_dim
        
        self.regularizer = tf.keras.regularizers.L2(weight_decay)
        
        self.lambda1 = lambda1
        
        self.lambda2 = lambda2
        
        self.eps = 1e-10
        
        self.encode_layers = []
        
        self.decode_layers = []
        
        self.share_layers_encoder = []
        
        self.share_layers_decoder = [] 
        
        for i in range (len(encode_dim)):#Encoding the data 
            
            self.encode_layers.append(Dense(int(encode_dim[i]), kernel_regularizer = self.regularizer,
            trainable = True))
            
            self.encode_layers.append(tf.keras.layers.LeakyReLU(alpha = 0.5))
            
        for j in range(len(share_dim)):#the share layer
             
            self.share_layers_encoder.append(Dense(int(share_dim[j]), kernel_regularizer = self.regularizer,
            trainable = True))
            
            self.share_layers_encoder.append(tf.keras.layers.LeakyReLU(alpha = 0.5))
        
        #the latent distrubution generator #Note z_dim controls the number of columns in the latent distribution  
        self.z_mu_layer = Dense(z_dim, activation = 'tanh', kernel_regularizer = self.regularizer)
        
        self.z_sigma_layer = Dense(z_dim, activation = 'tanh', kernel_regularizer = self.regularizer)
        
        for k in range(len(share_dim)):#the share layer
             
            self.share_layers_decoder.append(Dense(int(share_dim[k]), kernel_regularizer = self.regularizer,
            trainable = True))
            
            self.share_layers_decoder.append(tf.keras.layers.LeakyReLU(alpha = 0.5))     
        
        for l in range(len(decode_dim)): #The decoder/generator
            
            if l != len(decode_dim)-1:
            
                self.decode_layers.append(Dense(int(decode_dim[l]), kernel_regularizer = self.regularizer
                        ,trainable = True))
            
                self.decode_layers.append(tf.keras.layers.LeakyReLU(alpha = 0.5)) 
                
            else:

                self.decode_layers.append(Dense(int(decode_dim[l]), activation = 'sigmoid',
                    kernel_regularizer = self.regularizer,trainable = True))
            
            #Note the last value of decode_dim controls the number of columns in the reconstructed matrix   
        
    def call (self, interaction_matrix):
                
        for encode_layer in self.encode_layers:
            
            interaction_matrix = encode_layer(interaction_matrix)
        
        for share_layer_encoder in self.share_layers_encoder:
            
            interaction_matrix = share_layer_encoder(interaction_matrix)
        
        self.z_mean = self.z_mu_layer(interaction_matrix)
        
        self.z_sd = self.z_sigma_layer(interaction_matrix)
        
        e = tf.random.normal(tf.shape(self.z_sd))
            
        interaction_matrix = self.z_mean + tf.sqrt(tf.maximum(tf.exp(self.z_sd), self.eps)) * e
        
        z = tf.identity(interaction_matrix)#Z_A or Z_B 
        
        for share_layer_decoder in self.share_layers_decoder[::-1]:
            
            interaction_matrix = share_layer_decoder(interaction_matrix)
        
        for decode_layer in self.decode_layers:

            interaction_matrix = decode_layer(interaction_matrix)
        
        self.interaction_matrix_recon = tf.identity(interaction_matrix)
        
        return interaction_matrix, z
    
    def vae_loss(self, interaction_matrix):
        
        loss_kl = self.lambda1 * kl_loss(self.z_mean, self.z_sd)
    
        reconstruct_loss = self.lambda2 * loss_reconstruct(interaction_matrix, self.interaction_matrix_recon)
    
        loss = loss_kl + reconstruct_loss
    
        return loss 
    
    def get_config(self):
        
        configuration = super(cycle_consistency, self).get_config()
        
        configuration.update({
            'lambda1':self.lambda1,'lambda2':self.lambda2,'regularizer':self.regularizer,'eps':self.eps,
            'encode_layers':[encode_layer.get_config() for encode_layer in self.encode_layers],
            'share_layers_encoder':[e_share_layer.get_config() for e_share_layer in self.share_layers_encoder],
            'decode_layers':[decode_layer.get_config() for decode_layer in self.decode_layers],
            'share_layers_decoder':[d_layer_decoder.get_config() for d_layer_decoder in self.share_layers_decoder],     
        })
        
        return configuration

In [11]:
def load_and_test(domainA_test, domainB_test, file_path_VAE_A, file_path_VAE_B):

    VAE_A = tf.saved_model.load(file_path_VAE_A)
    
    VAE_B = tf.saved_model.load(file_path_VAE_B)
    
    y_AA = tf.zeros(domainA_test.shape, dtype = tf.float32)
    
    y_BB = tf.zeros(domainB_test.shape, dtype = tf.float32)
    
    i = 0
    
    for user_vector_A in domainA_test:
    
        user_vector_A = tf.reshape(user_vector_A, (1,domainA_test.shape[1]))
    
        x_AA,Z_A = VAE_A(user_vector_A)
        
        y_AA = tf.tensor_scatter_nd_update(y_AA, [[i]], x_AA)
        
        i += 1 
        
    j = 0
    
    for user_vector_B in domainB_test:
    
        user_vector_B = tf.reshape(user_vector_B, (1,domainB_test.shape[1]))
    
        x_BB,Z_B = VAE_B(user_vector_B)
        
        y_BB = tf.tensor_scatter_nd_update(y_BB, [[j]], x_BB)
        
        j += 1
    
    return y_AA, y_BB

In [12]:
def build_and_train_Model(encode_dim_A, share_dim, z_dim, decode_dim_A, training_dataset_A, encode_dim_B,
                        decode_dim_B, training_dataset_B, weight_decay, lambda1, lambda2, learning_rate_A, 
                        learning_rate_B, num_epochs_A, num_epochs_B, batch_size_A, batch_size_B,
                        file_path_VAE_A,file_path_VAE_B):
    
    #defining the VAEs for domain A and B 
    
    VAE_A = VAE(encode_dim_A, share_dim, z_dim, decode_dim_A, training_dataset_A, lambda1, lambda2, weight_decay)
    
    VAE_B = VAE(encode_dim_B, share_dim, z_dim, decode_dim_B, training_dataset_B, lambda1, lambda2, weight_decay)
    
    #training VAE_A, VAE_B
    
    optimizer_A = tf.keras.optimizers.legacy.Adam(learning_rate_A)
    
    number_of_batches_A = len(training_dataset_A)//batch_size_A
    
    number_of_batches_B = len(training_dataset_B)//batch_size_B
    
    x1 = []
    y1 = []
    
    print("VAE_A training: \n")
    
    for epoch in range(num_epochs_A):
        
        print('epoch number: ', epoch + 1)
        
        x1.append(epoch + 1)
        
        for batch_number in range(number_of_batches_A):
            
            start_id = batch_number * batch_size_A
            
            end_id = start_id + batch_size_A
            
            if end_id > len(training_dataset_A):
                
                end_id = len(training_dataset_A) - 1
            
            batch_A = training_dataset_A[start_id : end_id]
            
            with tf.GradientTape() as tape:
                y_AA = VAE_A(batch_A)
                vae_loss = VAE_A.vae_loss(batch_A)
                
            gradients = tape.gradient(vae_loss, VAE_A.trainable_variables)
            
            optimizer_A.apply_gradients(zip(gradients, VAE_A.trainable_variables))
            
        y1.append(vae_loss)    
        
        print('Training Set VAE loss: ', vae_loss)
        
    optimizer_B = tf.keras.optimizers.legacy.Adam(learning_rate_B)
    
    x2 = []
    y2 = []
    
    print("\n VAE_B training: \n")
    
    for epoch in range(num_epochs_B):
        
        print('epoch number: ', epoch + 1)
        
        x2.append(epoch+1)
        
        for batch_number in range(number_of_batches_B):
            
            start_id = batch_number * batch_size_B
            
            end_id = start_id + batch_size_B
            
            if end_id > len(training_dataset_B):
                
                end_id = len(training_dataset_B) - 1
            
            batch_B = training_dataset_B[start_id : end_id]

            with tf.GradientTape() as tape:
                
                y_BB = VAE_B(batch_B)
                vae_loss = VAE_B.vae_loss(batch_B)
                
            gradients = tape.gradient(vae_loss, VAE_B.trainable_variables)
            
            optimizer_B.apply_gradients(zip(gradients, VAE_B.trainable_variables))
            
        y2.append(vae_loss)    
        
        print('Training Set VAE loss: ', vae_loss)
        
    tf.saved_model.save(VAE_A, file_path_VAE_A)
        
    tf.saved_model.save(VAE_B, file_path_VAE_B)
        
    return x1,y1,x2,y2

In [13]:
x1,y1,x2,y2= build_and_train_Model([200], [100], 50, [200] , domainA_train, [200], [200], 
                     domainB_train, 0.01, 0.1, 0.2, 1e-5, 1e-5 , 200, 200, 30, 30, 
                    '/Users/ahmedaly/Desktop/Final_Model/VAE_A', '/Users/ahmedaly/Desktop/Final_Model/VAE_B')

y_AA, y_BB = load_and_test(domainA_test, domainB_test, '/Users/ahmedaly/Desktop/Final_Model/VAE_A', 
              '/Users/ahmedaly/Desktop/Final_Model/VAE_B')

#ML1M settings
#x1,y1,x2,y2 = build_and_train_Model([200], [100], 50, [200] , domainA_train_ML1M, [100], [200], 
                     # domainB_train_ML1M, 0.01, 0.1, 0.2, 1e-4, 1e-4 , 30, 20, 30, 30,
                     #'/Users/ahmedaly/Desktop/Final_Model/VAE_A', '/Users/ahmedaly/Desktop/Final_Model/VAE_B') 

#ML100K settings
#x1,y1,x2,y2= build_and_train_Model([200], [100], 50, [200] , domainA_train, [200], [200], 
                     # domainB_train, 0.01, 0.1, 0.2, 1e-5, 1e-5 , 200, 200, 30, 30, 
                      #'/Users/ahmedaly/Desktop/Final_Model/VAE_A', '/Users/ahmedaly/Desktop/Final_Model/VAE_B')

VAE_A training: 

epoch number:  1
Training Set VAE loss:  tf.Tensor(43.604, shape=(), dtype=float32)
epoch number:  2
Training Set VAE loss:  tf.Tensor(43.558273, shape=(), dtype=float32)
epoch number:  3
Training Set VAE loss:  tf.Tensor(43.559116, shape=(), dtype=float32)
epoch number:  4
Training Set VAE loss:  tf.Tensor(43.504074, shape=(), dtype=float32)
epoch number:  5
Training Set VAE loss:  tf.Tensor(43.526756, shape=(), dtype=float32)
epoch number:  6
Training Set VAE loss:  tf.Tensor(43.533764, shape=(), dtype=float32)
epoch number:  7
Training Set VAE loss:  tf.Tensor(43.50321, shape=(), dtype=float32)
epoch number:  8
Training Set VAE loss:  tf.Tensor(43.42007, shape=(), dtype=float32)
epoch number:  9
Training Set VAE loss:  tf.Tensor(43.41777, shape=(), dtype=float32)
epoch number:  10
Training Set VAE loss:  tf.Tensor(43.360268, shape=(), dtype=float32)
epoch number:  11
Training Set VAE loss:  tf.Tensor(43.355927, shape=(), dtype=float32)
epoch number:  12
Training Se

Training Set VAE loss:  tf.Tensor(40.317104, shape=(), dtype=float32)
epoch number:  96
Training Set VAE loss:  tf.Tensor(40.24119, shape=(), dtype=float32)
epoch number:  97
Training Set VAE loss:  tf.Tensor(40.12828, shape=(), dtype=float32)
epoch number:  98
Training Set VAE loss:  tf.Tensor(40.146637, shape=(), dtype=float32)
epoch number:  99
Training Set VAE loss:  tf.Tensor(40.039696, shape=(), dtype=float32)
epoch number:  100
Training Set VAE loss:  tf.Tensor(40.226936, shape=(), dtype=float32)
epoch number:  101
Training Set VAE loss:  tf.Tensor(40.130257, shape=(), dtype=float32)
epoch number:  102
Training Set VAE loss:  tf.Tensor(40.302658, shape=(), dtype=float32)
epoch number:  103
Training Set VAE loss:  tf.Tensor(40.241486, shape=(), dtype=float32)
epoch number:  104
Training Set VAE loss:  tf.Tensor(40.28061, shape=(), dtype=float32)
epoch number:  105
Training Set VAE loss:  tf.Tensor(40.296547, shape=(), dtype=float32)
epoch number:  106
Training Set VAE loss:  tf.T

Training Set VAE loss:  tf.Tensor(39.736275, shape=(), dtype=float32)
epoch number:  189
Training Set VAE loss:  tf.Tensor(39.782597, shape=(), dtype=float32)
epoch number:  190
Training Set VAE loss:  tf.Tensor(39.74953, shape=(), dtype=float32)
epoch number:  191
Training Set VAE loss:  tf.Tensor(39.805393, shape=(), dtype=float32)
epoch number:  192
Training Set VAE loss:  tf.Tensor(39.824448, shape=(), dtype=float32)
epoch number:  193
Training Set VAE loss:  tf.Tensor(39.78998, shape=(), dtype=float32)
epoch number:  194
Training Set VAE loss:  tf.Tensor(39.8174, shape=(), dtype=float32)
epoch number:  195
Training Set VAE loss:  tf.Tensor(39.81648, shape=(), dtype=float32)
epoch number:  196
Training Set VAE loss:  tf.Tensor(39.78226, shape=(), dtype=float32)
epoch number:  197
Training Set VAE loss:  tf.Tensor(39.809326, shape=(), dtype=float32)
epoch number:  198
Training Set VAE loss:  tf.Tensor(39.75731, shape=(), dtype=float32)
epoch number:  199
Training Set VAE loss:  tf.T

Training Set VAE loss:  tf.Tensor(28.734009, shape=(), dtype=float32)
epoch number:  83
Training Set VAE loss:  tf.Tensor(28.690762, shape=(), dtype=float32)
epoch number:  84
Training Set VAE loss:  tf.Tensor(28.53637, shape=(), dtype=float32)
epoch number:  85
Training Set VAE loss:  tf.Tensor(28.573214, shape=(), dtype=float32)
epoch number:  86
Training Set VAE loss:  tf.Tensor(28.538296, shape=(), dtype=float32)
epoch number:  87
Training Set VAE loss:  tf.Tensor(28.504436, shape=(), dtype=float32)
epoch number:  88
Training Set VAE loss:  tf.Tensor(28.630611, shape=(), dtype=float32)
epoch number:  89
Training Set VAE loss:  tf.Tensor(28.581467, shape=(), dtype=float32)
epoch number:  90
Training Set VAE loss:  tf.Tensor(28.594307, shape=(), dtype=float32)
epoch number:  91
Training Set VAE loss:  tf.Tensor(28.497936, shape=(), dtype=float32)
epoch number:  92
Training Set VAE loss:  tf.Tensor(28.50184, shape=(), dtype=float32)
epoch number:  93
Training Set VAE loss:  tf.Tensor(

Training Set VAE loss:  tf.Tensor(27.841358, shape=(), dtype=float32)
epoch number:  177
Training Set VAE loss:  tf.Tensor(27.88733, shape=(), dtype=float32)
epoch number:  178
Training Set VAE loss:  tf.Tensor(27.876482, shape=(), dtype=float32)
epoch number:  179
Training Set VAE loss:  tf.Tensor(27.867191, shape=(), dtype=float32)
epoch number:  180
Training Set VAE loss:  tf.Tensor(27.875927, shape=(), dtype=float32)
epoch number:  181
Training Set VAE loss:  tf.Tensor(27.795774, shape=(), dtype=float32)
epoch number:  182
Training Set VAE loss:  tf.Tensor(27.85625, shape=(), dtype=float32)
epoch number:  183
Training Set VAE loss:  tf.Tensor(27.854641, shape=(), dtype=float32)
epoch number:  184
Training Set VAE loss:  tf.Tensor(27.829926, shape=(), dtype=float32)
epoch number:  185
Training Set VAE loss:  tf.Tensor(27.888683, shape=(), dtype=float32)
epoch number:  186
Training Set VAE loss:  tf.Tensor(27.815926, shape=(), dtype=float32)
epoch number:  187
Training Set VAE loss: 

2023-07-26 02:43:11.673075: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


In [14]:
import numpy as np
import math as m

In [15]:
def user_top_predictions(y, Rating_Matrix, k):#y is a tensor, and dataframe is its associated dataframe
    
    user_highest_predicted_movies = np.empty((y.shape[0], k))
    
    MovieIDs = Rating_Matrix.columns.tolist()
    
    list_UserIDs = Rating_Matrix.index.tolist()
    
    i = 0
    
    all_Movie_indices = np.empty((y.shape[0], k))
    
    UserIDs = np.empty(y.shape[0])
    
    for click_vector in y:
        
        highest_predicted_scores, indices_of_highest_scores = tf.math.top_k(click_vector, k)
        
        UserIDs[i] = list_UserIDs[i]
        
        Movie_indices = [MovieIDs[id] for id in indices_of_highest_scores]
    
        user_highest_predicted_movies[i] = Movie_indices
        
        all_Movie_indices[i] = indices_of_highest_scores
        
        i += 1
    
    all_Movie_indices = [[int(x) for x in row] for row in all_Movie_indices]
    
    return user_highest_predicted_movies, all_Movie_indices, UserIDs

In [16]:
def calc_recall(top_k_movies_IDs, Rating_Matrix, all_Movie_indices):
        
    recall = 0
    
    for i in range(len(top_k_movies_IDs)):
        
        MovieIDs = top_k_movies_IDs[i]
        
        columns_greater_than_threshold = Rating_Matrix.iloc[i] > 3
        
        number_of_relevant_items = columns_greater_than_threshold.sum()
        
        if number_of_relevant_items == 0:
            
            continue
            
        columns_greater_than_threshold_2 = Rating_Matrix.iloc[i , all_Movie_indices[i]] > 3
        
        recommended_relevant_items = columns_greater_than_threshold_2.sum()
        
        user_recall = recommended_relevant_items/number_of_relevant_items
        
        recall = recall + user_recall 
    
    recall = recall/len(top_k_movies_IDs)
         
    return recall 

In [17]:
def calc_precision(top_k_movies_IDs, Rating_Matrix, all_Movie_indices):
    
    k = top_k_movies_IDs.shape[1]
    
    precision = 0
    
    for i in range(len(top_k_movies_IDs)):
        
        user_movie_indices = all_Movie_indices[i]
        
        recommended_relevant_movies = Rating_Matrix.iloc[i, user_movie_indices] > 3
        
        number_recommended_relevant_movies = recommended_relevant_movies.sum()
        
        user_precision = number_recommended_relevant_movies/k
        
        precision = precision + user_precision
    
    precision = precision/len(top_k_movies_IDs)
    
    return precision

In [18]:
def calc_NDCG(all_Movie_indices, Rating_Matrix):
        
    DCG = 0
    
    IDCG = 0
    
    NDCG = 0
    
    for i in range(len(all_Movie_indices)):
        
        row = all_Movie_indices[i]
        
        for j in row:
            
            relevance = Rating_Matrix.iloc[i,j]
            
            user_dcg = relevance/m.log(j + 2, 2)
            
            DCG = DCG + user_dcg
    
    for k in range(len(all_Movie_indices)):
        
        df = Rating_Matrix.iloc[k,all_Movie_indices[k]]
        
        row_arr = np.array(df)
        
        row_arr = sorted(row_arr, reverse = True)
        
        for l in range(len(row_arr)):
            
            user_idcg = row_arr[l]/m.log(l + 2, 2)
            
            IDCG = IDCG + user_idcg
    
    NDCG = DCG/IDCG
    
    return NDCG  

In [19]:
def Average_MSE (Rating_Matrix, y):#go back and modify the reconstruction loss to MSE in the loss function (maybe)
    
    rating_matrix = Rating_Matrix.to_numpy()
    
    y_numpy = y.numpy()
    
    MSE_arr = np.square(rating_matrix - y_numpy)
    
    MSE = np.sum(MSE_arr) / rating_matrix.size
    
    return MSE

In [20]:
print("Recall AA: \n")
for k in range(36, 361, 36):

    user_highest_predicted_movies_AA, all_Movie_indices_AA, user_IDs_AA = user_top_predictions(y_AA, DomainA_test_1, k)
    recall_AA = calc_recall(user_highest_predicted_movies_AA, DomainA_test_1, all_Movie_indices_AA)
    
    print(recall_AA)

print("NDCG AA: \n")
for k in range(36, 361, 36):
    user_highest_predicted_movies_AA, all_Movie_indices_AA, user_IDs_AA = user_top_predictions(y_AA, DomainA_test_1, k)
    NDCG_AA = calc_NDCG(all_Movie_indices_AA, DomainA_test_1) 

    print(NDCG_AA)

Recall AA: 

0.3661440919645031
0.577096845770239
0.6974443289060134
0.7780790106724497
0.8213574703787098
0.8552197251850623
0.8990957995802563
0.9300997868313318
0.9494010536971972
0.9598478973737568
NDCG AA: 

0.4160379879547238
0.49252048256457204
0.5233985709327116
0.5411832480065166
0.5580655412193836
0.5664880524844443
0.5708976742106414
0.5741563594105837
0.5766020128702879
0.5781432720861873


In [21]:
print("Precision AA: \n")

for k in range(5, 51, 5):
    
    user_highest_predicted_movies_AA, all_Movie_indices_AA, user_IDs_AA = user_top_predictions(y_AA, DomainA_test_1, k)
    precision_AA = calc_precision(user_highest_predicted_movies_AA, DomainA_test_1, all_Movie_indices_AA)

    #print('\nFor k =, ', k)
    #print('Precision for AA:', precision_AA)
    print(precision_AA)

Precision AA: 

0.28865248226950385
0.26134751773049647
0.2527186761229319
0.23865248226950334
0.22709219858155996
0.21501182033096916
0.20678824721377934
0.20017730496453898
0.19637509850275833
0.19170212765957442


In [22]:
print("Recall BB: \n")
for k in range(25, 251, 25):

    user_highest_predicted_movies_BB, all_Movie_indices_BB, user_IDs_BB = user_top_predictions(y_BB, DomainB_test_1, k)
    recall_BB = calc_recall(user_highest_predicted_movies_BB, DomainB_test_1, all_Movie_indices_BB)
    
    print(recall_BB)

print("NDCG BB: \n")
for k in range(25, 251, 25):
    user_highest_predicted_movies_BB, all_Movie_indices_BB, user_IDs_BB = user_top_predictions(y_BB, DomainB_test_1, k)
    NDCG_BB = calc_NDCG(all_Movie_indices_BB, DomainB_test_1) 

    print(NDCG_BB)

Recall BB: 

0.28758044543936473
0.4762260093090494
0.5961767230271268
0.7124741152617787
0.7791587189548836
0.8248474829156656
0.8556530627640693
0.8832676005056639
0.9118228417302743
0.9273507729101311
NDCG BB: 

0.47453684733945706
0.5252758137297806
0.5474362343558469
0.5610914107534194
0.5753185223458219
0.5825015389212249
0.587279227205097
0.5900664124460667
0.5905773844397425
0.5919695825177406


In [23]:
print("Precision BB: \n")

for k in range(5, 51, 5):
    
    user_highest_predicted_movies_BB, all_Movie_indices_BB, user_IDs_BB = user_top_predictions(y_BB, DomainB_test_1, k)
    precision_BB = calc_precision(user_highest_predicted_movies_BB, DomainB_test_1, all_Movie_indices_BB)

    #print('\nFor k =, ', k)
    #print('Precision for BB:', precision_BB)
    print(precision_BB)

Precision BB: 

0.21773049645390108
0.1975177304964541
0.18510638297872395
0.1739361702127658
0.16595744680851024
0.1607565011820333
0.15521783181357665
0.15106382978723393
0.14728132387706877
0.14262411347517734


In [24]:
MSE_AA = Average_MSE(DomainA_test_2, y_AA)
MSE_BB = Average_MSE(DomainB_test_2, y_BB)

In [25]:
print('Average MSE_AA: ', MSE_AA)
print('Average MSE_BB: ', MSE_BB)

Average MSE_AA:  0.22343945461405784
Average MSE_BB:  0.22744421321719316


In [26]:
def inject_fake_interactions(interaction_matrix):
    
    injected_interaction_matrix = interaction_matrix.copy()
    
    for column in interaction_matrix.columns:
        
        injected_interaction_matrix[column] = interaction_matrix[column].apply(
            lambda x: 1 - x if np.random.random() > 0.1 else x)
    
    fake_interaction_matrix = tf.cast(injected_interaction_matrix, tf.float32)
    
    return fake_interaction_matrix

#np.random.random() > 0.1 for ML1M 
#np.random.random() > 0.5 for ML100K

In [27]:
domainA_test_injected = inject_fake_interactions(DomainA_test_2)

domainB_test_injected = inject_fake_interactions(DomainB_test_2)

y_AA_injected, y_BB_injected = load_and_test(domainA_test_injected, domainB_test_injected, '/Users/ahmedaly/Desktop/Final_Model/VAE_A', 
              '/Users/ahmedaly/Desktop/Final_Model/VAE_B')

In [28]:
MSE_AA_injected = Average_MSE(DomainA_test_2, y_AA_injected)
MSE_BB_injected = Average_MSE(DomainB_test_2, y_BB_injected)

In [29]:
print('Average MSE_AA: ', MSE_AA_injected)
print('Average MSE_BB: ', MSE_BB_injected)

Average MSE_AA:  0.24325357019612348
Average MSE_BB:  0.2566139991085023
